# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features_3h.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(15059, 11841)


12h_dft_dom_cycle_ddt_lag39    362
12h_dft_dom_cycle_ddt_lag38    358
12h_dft_dom_cycle_dt_lag39     358
12h_dft_dom_cycle_lag39        354
12h_dft_dom_cycle_ddt_lag37    354
                              ... 
3h_conv_23                       0
3h_conv_24                       0
3h_adaptive_stochastic           0
3h_trendflex                     0
3h_vwap                          0
Length: 11841, dtype: int64

In [2]:
import numpy as np
from jesse import utils

label = np.load("data/label_side.npy")
df_label = utils.numpy_candles_to_dataframe(label[:, :6]).set_index("date")
df_label["side_label"] = label[:, 6]
df_label.head(1)

,open,close,high,low,volume,side_label
date,,,,,,
2020-01-02 10:47:00,7157.37,7289.98,7379.0,6863.44,221242.464,1.0


In [3]:
side_features = df_features.iloc[362:]
side_label = df_label.iloc[362:]

side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)
side_label = side_label["side_label"].to_numpy()

print(side_features.shape)
print(side_label.shape)

side_features.isna().sum(axis=0).sort_values(ascending=False)

(14697, 11841)
(14697,)


/var/folders/h9/d_xpszfn6yz9r8zww450m44w0000gn/T/ipykernel_79784/3052797767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_label["side_label"] = (side_label["side_label"].astype(int) == 1).astype(int)


12h_ac_0                    0
45m_ac_4                    0
3h_williams_r_ddt_lag35     0
3h_williams_r_ddt_lag36     0
3h_williams_r_ddt_lag37     0
                           ..
3h_ac_2                     0
3h_ac_3                     0
3h_ac_4                     0
3h_ac_5                     0
45m_williams_r_ddt_lag39    0
Length: 11841, dtype: int64

In [4]:
from custom_indicators.toolbox.feature_selection.fcq_selector import FCQSelector

selector = FCQSelector(max_features=3000)
selector.fit(side_features, side_label)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: 3h_williams_r (最大F值: 7428.3349)
➤ 计算特征冗余度...
➤ 总计选择3000个特征 (已选择1个，还需选择2999个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2999/2999 [28:18<00:00,  1.77特征/s]



✅ 特征选择完成：从11841个特征中选择了3000个，舍弃了8841个
✅ 选择的特征: ['3h_williams_r', '12h_adaptive_rsi_ddt_lag2', '3h_hurst_coef_fast_ddt_lag16', '3h_acc_swing_index_dt', '45m_williams_r', '3h_williams_r_lag1', '12h_adaptive_stochastic_lag7', '12h_hurst_coef_slow_lag5', '3h_trendflex_dt', '3h_fisher', '3h_fisher_dt', '3h_trendflex', '45m_trendflex', '3h_natr_dt', '3h_vwap_dt', '3h_voss', '3h_williams_r_dt', '3h_evenbetter_sinewave_short', '45m_williams_r_lag1', '12h_acc_swing_index_dt', '12h_williams_r', '45m_evenbetter_sinewave_long', '45m_fisher', '3h_evenbetter_sinewave_long', '45m_trendflex_lag1', '3h_stc', '12h_williams_r_dt', '3h_stc_dt', '3h_williams_r_lag2', '3h_dft_dom_cycle_lag3', '3h_acc_swing_index_dt_lag1', '45m_williams_r_lag2', '45m_evenbetter_sinewave_long_lag1', '3h_fisher_lag1', '3h_roofing_filter_ddt', '45m_fisher_lag1', '12h_comb_spectrum_pwr_17', '45m_trendflex_lag2', '45m_evenbetter_sinewave_short', '3h_voss_dt', '12h_mod_stochastic_dt_lag27', '3h_trendflex_lag1', '45m_evenbetter_sin

In [5]:
side_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
side_res

3h_williams_r                            7.428335e+03
3h_williams_r_lag1                       3.409076e+03
3h_acc_swing_index_dt                    2.742781e+03
3h_fisher                                2.640753e+03
45m_trendflex                            2.626052e+03
                                             ...     
45m_adaptive_stochastic_ddt_lag5         7.666567e-07
45m_ehlers_early_onset_trend_ddt_lag2    6.995421e-07
3h_mod_rsi_ddt_lag17                     3.330427e-07
12h_ac_45                                2.202565e-08
3h_natr_ddt_lag24                        4.682362e-09
Length: 11841, dtype: float64

In [6]:
import json

with open(f"data/side_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(side_res.to_dict(), f, indent=4)

In [10]:
import json

import pandas as pd

with open(f"data/side_features_FCQSelector.json", "r") as f:
    side_res = pd.Series(json.load(f))

In [8]:
DOLLAR_BAR_SHORT_TERM = "45m"
DOLLAR_BAR_MEDIUM_TERM = "3h"
DOLLAR_BAR_LONG_TERM = "12h"

feature_info = {
    "side": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
    },
    "meta": {
        DOLLAR_BAR_SHORT_TERM: [],
        DOLLAR_BAR_MEDIUM_TERM: [],
        DOLLAR_BAR_LONG_TERM: [],
        "model_res": [],
    },
}

for k, v in side_res[:3000].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["side"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["side"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["side"][DOLLAR_BAR_LONG_TERM].append(k)

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("model/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_3h.parquet")
meta_label = np.load("data/label_meta.npy")

side_model = lgb.Booster(model_file="model/model_side.txt")
side_model_pred_prob = side_model.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_pred_prob

assert df_features.shape[0] == meta_label.shape[0]
assert "side_model_res" in df_features.columns

df_features.isna().sum(axis=0).sort_values(ascending=False)

12h_dft_dom_cycle_ddt_lag39    362
12h_dft_dom_cycle_ddt_lag38    358
12h_dft_dom_cycle_dt_lag39     358
12h_dft_dom_cycle_lag39        354
12h_dft_dom_cycle_ddt_lag37    354
                              ... 
3h_comb_spectrum_pwr_14          0
3h_comb_spectrum_pwr_13          0
3h_comb_spectrum_pwr_12          0
3h_comb_spectrum_pwr_11          0
side_model_res                   0
Length: 11842, dtype: int64

In [2]:
meta_features = df_features[362:]
meta_label = meta_label[362:]

meta_label = pd.Series(meta_label[:, 6].astype(int), index=meta_features.index)

# meta_features = meta_features[meta_label["ret"].notna()]
# meta_label = meta_label[meta_label["ret"].notna()]["bin"]

print(meta_features.shape)
print(meta_label.shape)

meta_features.isna().sum(axis=0).sort_values(ascending=False)

(14697, 11842)
(14697,)


12h_ac_0                   0
45m_ac_4                   0
3h_williams_r_ddt_lag35    0
3h_williams_r_ddt_lag36    0
3h_williams_r_ddt_lag37    0
                          ..
3h_ac_2                    0
3h_ac_3                    0
3h_ac_4                    0
3h_ac_5                    0
side_model_res             0
Length: 11842, dtype: int64

In [3]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.fcq_selector import FCQSelector

selector = FCQSelector(max_features=3000)
selector.fit(meta_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性...
✓ 选择第1个特征: 12h_vwap (最大F值: 89.5742)
➤ 计算特征冗余度...
➤ 总计选择3000个特征 (已选择1个，还需选择2999个)...
➤ 开始MRMR迭代选择过程...


选择特征: 100%|███████████████████████████████████████████████| 2999/2999 [10:15<00:00,  4.87特征/s]



✅ 特征选择完成：从11842个特征中选择了3000个，舍弃了8842个
✅ 选择的特征: ['12h_vwap', '3h_dft_dom_cycle_ddt_lag26', '12h_adaptive_cci_ddt_lag32', '3h_homodyne_ddt_lag17', '3h_mod_stochastic_dt_lag37', '3h_natr_ddt', '12h_hurst_coef_fast_dt_lag8', '3h_comb_spectrum_dom_cycle_dt_lag11', '45m_comb_spectrum_dom_cycle_ddt_lag36', '45m_stc_ddt_lag22', '45m_homodyne_dt_lag22', '3h_forecast_oscillator_lag33', '45m_natr_ddt_lag28', '45m_adaptive_rsi_ddt_lag39', '3h_dual_diff_dt_lag28', '3h_ac_22', '3h_acc_swing_index_lag7', '3h_fisher_dt', '12h_vwap_ddt_lag15', '45m_homodyne_lag4', '45m_dft_dom_cycle_dt_lag26', '12h_adaptive_cci_ddt_lag21', '3h_vwap_lag2', '12h_hurst_coef_fast_ddt_lag22', '3h_fisher_ddt_lag20', '12h_adaptive_bp_lead_ddt_lag39', '12h_dft_spectrum_5', '12h_evenbetter_sinewave_short_ddt_lag24', '45m_vwap_ddt_lag15', '3h_evenbetter_sinewave_short_dt_lag36', '3h_vwap_lag3', '3h_evenbetter_sinewave_short_ddt_lag17', '3h_comb_spectrum_dom_cycle_ddt_lag39', '12h_ehlers_early_onset_trend_ddt_lag23', '12h_william

In [4]:
import json

with open(f"data/meta_features_{selector.__class__.__name__}.json", "w") as f:
    json.dump(meta_res.to_dict(), f, indent=4)

In [5]:
import json

import pandas as pd

with open("data/meta_features_FCQSelector.json", "r") as f:
    meta_res = pd.Series(json.load(f))

In [6]:
meta_res.to_dict()["side_model_res"]

4.891870718921258

In [7]:
from pathlib import Path

feature_info = Path("model/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

DOLLAR_BAR_SHORT_TERM = "45m"
DOLLAR_BAR_MEDIUM_TERM = "3h"
DOLLAR_BAR_LONG_TERM = "12h"


feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = []
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = []
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = []
feature_info["meta"]["model_res"] = []

for k, v in meta_res[:3000].to_dict().items():
    if k.startswith(f"{DOLLAR_BAR_SHORT_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_SHORT_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_MEDIUM_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM].append(k)
    elif k.startswith(f"{DOLLAR_BAR_LONG_TERM}_"):
        feature_info["meta"][DOLLAR_BAR_LONG_TERM].append(k)
    else:
        feature_info["meta"]["model_res"].append(k)

feature_info["side"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["side"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["side"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["side"][DOLLAR_BAR_LONG_TERM])),
)

feature_info["meta"][DOLLAR_BAR_SHORT_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_SHORT_TERM])),
)
feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_MEDIUM_TERM])),
)
feature_info["meta"][DOLLAR_BAR_LONG_TERM] = sorted(
    list(set(feature_info["meta"][DOLLAR_BAR_LONG_TERM])),
)
feature_info["meta"]["model_res"] = sorted(
    list(set(feature_info["meta"]["model_res"])),
)

with open("model/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)